In [1]:
import numpy as np
import pandas as pd

In [39]:
#1.1
modules_mass = pd.read_fwf("./modules_mass.txt", header=None)

In [52]:
def get_modules_fuel(modules_mass):
    total_fuel = 0
    for mass in modules_mass[0]:
        total_fuel += int(mass/3) -2
    return total_fuel
get_modules_fuel(modules_mass)

3374289

In [58]:
#1.2
def get_total_fuel(modules_mass):
    total_fuel = 0
    for mass in modules_mass[0]:
        module_fuel = int(mass/3) - 2
        total_fuel += module_fuel + get_fuel_of_fuel(module_fuel)
    return total_fuel

def get_fuel_of_fuel(fuel):
    new_fuel = int(fuel/3) - 2
    if new_fuel <= 0: return 0
    return new_fuel + fuel_of_fuel(new_fuel)
get_total_fuel(modules_mass)

In [192]:
#2.1
grav_assist_prog = np.loadtxt("./grav_assist_prog.txt", delimiter=",").astype(int)
grav_assist_prog[1] = 12
grav_assist_prog[2] = 2

In [ ]:
def computer(code):
    for i in range( 0, len(code), 4 ):
        optcode = code[i]
        a = code[i+1]
        b = code[i+2]
        c = code[i+3]
        if optcode == 99: 
            return code
        elif optcode == 1:
            code[c] = code[a] + code[b]
        elif optcode == 2:
            code[c] = code[a] * code[b]
        else: return "Unknown optcode"
computer( grav_assist_prog.copy() )

In [186]:
#2.2
def find_noun_verb():
    grav_assist_prog_copy = grav_assist_prog.copy()
    for i in range(0,100):
        for j in range(0,100):
            grav_assist_prog_copy = grav_assist_prog.copy()
            grav_assist_prog_copy[1] = i
            grav_assist_prog_copy[2] = j
            output = computer(grav_assist_prog_copy)[0]
            if output == 19690720: 
                return i,j
    return False
find_noun_verb()